# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [136]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [137]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [138]:
# confirm that our new database was created
db = mongo['uk_food']

In [139]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [140]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [141]:
# Create a dictionary for the new restaurant data
new_restaurant = new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [142]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('67e219d2f365b9cce7fa5be7'), acknowledged=True)

In [143]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "New Restaurant"}))

{'BusinessName': 'New Restaurant',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': ['BusinessType'],
 'LocalAuthorityName': 'Test Authority',
 'RatingValue': 5,
 '_id': ObjectId('67e0378a5fd3e2306c2e2940'),
 'geocode': {'latitude': 0.0, 'longitude': 0.0}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [144]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

business_type_id = result["BusinessTypeID"]

establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)


UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [145]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "New Restaurant"},
    {"$set": {"BusinessTypeID": ["BusinessType"]}}
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [146]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "New Restaurant"}))

{'BusinessName': 'New Restaurant',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': ['BusinessType'],
 'LocalAuthorityName': 'Test Authority',
 'RatingValue': 5,
 '_id': ObjectId('67e0378a5fd3e2306c2e2940'),
 'geocode': {'latitude': 0.0, 'longitude': 0.0}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [147]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover: {dover_count}")


Number of establishments in Dover: 0


In [148]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [149]:
# Check if any remaining documents include Dover
dover_check = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Dover documents after delete: {dover_check}")

Dover documents after delete: 0


In [150]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())
establishments.update_many({}, [
    {"$set": {
        "geocode.longitude": {"$toDouble": "$geocode.longitude"},
        "geocode.latitude": {"$toDouble": "$geocode.latitude"}
    }}
])

{'BusinessName': 'New Restaurant',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': ['BusinessType'],
 'LocalAuthorityName': 'Test Authority',
 'RatingValue': 5,
 '_id': ObjectId('67e0378a5fd3e2306c2e2940'),
 'geocode': {'latitude': 0.0, 'longitude': 0.0}}


UpdateResult({'n': 38795, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [151]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{'$set': {"geocode.longitude": {'$toDecimal': "$geocode.longitude"}}}]
)
establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{'$set': {"geocode.latitude": {'$toDecimal': "$geocode.latitude"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [152]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [153]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{'$set': {"RatingValue": {'$toInt': "$RatingValue"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [154]:
# Check that the coordinates and rating value are now numbers
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Select the database and collection
db = client["uk_food"]
establishments = db["establishments"]

# Fetch one document (including fields that need checking)
doc = establishments.find_one({}, {"geocode.latitude": 1, "geocode.longitude": 1, "RatingValue": 1})

# Check if a document was found
if doc:
    print(doc)

    # Check data types explicitly
    print("Latitude Type:", type(doc.get("geocode", {}).get("latitude", None)))
    print("Longitude Type:", type(doc.get("geocode", {}).get("longitude", None)))
    print("RatingValue Type:", type(doc.get("RatingValue", None)))
else:
    print("No document found. Check if your database and collection exist.")


{'_id': ObjectId('67e0378a5fd3e2306c2e2940'), 'RatingValue': 5, 'geocode': {'longitude': 0.0, 'latitude': 0.0}}
Latitude Type: <class 'float'>
Longitude Type: <class 'float'>
RatingValue Type: <class 'int'>
